# MR-KG Backend API Demonstration

This notebook demonstrates the functionality of the MR-KG backend API endpoints.

## Prerequisites

1. Start the backend development server:
   ```bash
   cd backend
   just dev
   ```

2. Ensure the databases are properly configured (see `@docs/ENV.md`)

## Base URL

- Development: http://localhost:8000
- API Documentation: http://localhost:8000/docs

In [ ]:
import requests
import json
from typing import Any
from pprint import pprint

BASE_URL = "http://localhost:8000/api/v1"


def pretty_print(data: Any) -> None:
    """Pretty print JSON data."""
    print(json.dumps(data, indent=2))


def check_response(response: requests.Response) -> dict:
    """Check response status and return JSON data."""
    print(f"Status: {response.status_code}")
    if response.ok:
        return response.json()
    else:
        print(f"Error: {response.text}")
        return {}


## 1. Health Check Endpoints

Monitor system health and diagnostics.

### 1.1 Basic Health Check

In [ ]:
response = requests.get(f"{BASE_URL}/health/")
data = check_response(response)
pretty_print(data)


Status: 200
{
  "success": true,
  "timestamp": "2025-10-16T07:52:47.276103Z",
  "request_id": "7033c2eb-7e92-4885-a87d-976921d42252",
  "data": {
    "status": "healthy",
    "version": "0.1.0",
    "uptime": 414.28172993659973,
    "timestamp": "2025-10-16T07:52:47.275528Z"
  }
}


### 1.2 Detailed Health Check

In [ ]:
response = requests.get(f"{BASE_URL}/health/detailed")
data = check_response(response)
pretty_print(data)


Status: 200
{
  "status": "healthy",
  "timestamp": "2025-10-16T08:52:50.116168",
  "vector_store": {
    "database_path": "/Users/ik18445/local-projects/+dmer/+mr-paper-data-extraction/mr-kg/backend/../data/db/vector_store.db",
    "accessible": true,
    "table_count": 5,
    "view_count": 3,
    "index_count": 11,
    "last_checked": "2025-10-16T08:52:50.116168",
    "error": null
  },
  "trait_profile": {
    "database_path": "/Users/ik18445/local-projects/+dmer/+mr-paper-data-extraction/mr-kg/backend/../data/db/trait_profile_db.db",
    "accessible": true,
    "table_count": 2,
    "view_count": 3,
    "index_count": 6,
    "last_checked": "2025-10-16T08:52:50.116168",
    "error": null
  },
  "performance_metrics": {
    "vector_store_query_time": 0.00011706352233886719,
    "trait_profile_query_time": 8.082389831542969e-05,
    "complex_query_time": 0.009627819061279297
  }
}


### 1.3 Database Health Check

In [ ]:
response = requests.get(f"{BASE_URL}/health/database")
data = check_response(response)
pretty_print(data)


Status: 200
{
  "success": true,
  "timestamp": "2025-10-16T07:52:53.314096Z",
  "request_id": "f904e58a-8fa2-4100-a1bf-2f7f21ae1df5",
  "data": {
    "status": "healthy",
    "vector_store_connection": true,
    "trait_profile_connection": true,
    "pool_status": {
      "vector_store_connections": 3,
      "trait_profile_connections": 3,
      "max_connections": 10,
      "initialized": true
    }
  }
}


### 1.4 System Information

In [ ]:
response = requests.get(f"{BASE_URL}/health/system")
data = check_response(response)
pretty_print(data)


### 1.5 Application Metrics

In [ ]:
response = requests.get(f"{BASE_URL}/health/metrics")
data = check_response(response)
pretty_print(data)


Status: 200
{
  "success": true,
  "timestamp": "2025-10-16T07:52:56.364432Z",
  "request_id": "8a83df84-0ede-4b0a-953c-23bd5e59b85a",
  "data": {
    "requests_total": 0,
    "active_connections": 0,
    "database_queries": 0,
    "cache_hits": 0,
    "cache_misses": 0,
    "response_time_avg": 0.0
  }
}


### 1.6 Readiness and Liveness Probes

In [6]:
print("Readiness Check:")
response = requests.get(f"{BASE_URL}/health/ready")
data = check_response(response)
pretty_print(data)

print("\nLiveness Check:")
response = requests.get(f"{BASE_URL}/health/live")
data = check_response(response)
pretty_print(data)


Readiness Check:
Status: 200
{
  "success": true,
  "timestamp": "2025-10-16T07:53:07.034271Z",
  "request_id": "6229e90b-f3d0-431e-9011-7d081df230b4",
  "data": {
    "ready": true,
    "timestamp": "2025-10-16T07:53:07.034241+00:00",
    "checks": {
      "database": {
        "ready": true,
        "details": {
          "status": "healthy",
          "vector_store_connection": true,
          "trait_profile_connection": true,
          "pool_status": {
            "vector_store_connections": 3,
            "trait_profile_connections": 3,
            "max_connections": 10,
            "initialized": true
          }
        }
      },
      "database_operations": {
        "ready": true,
        "trait_count": 25385
      }
    }
  }
}

Liveness Check:
Status: 200
{
  "success": true,
  "timestamp": "2025-10-16T07:53:07.040748Z",
  "request_id": "8396a3af-ce18-44a3-bf98-cb9c0600db86",
  "data": {
    "alive": "true",
    "timestamp": "2025-10-16T07:53:07.040725+00:00",
    "uptime":

## 2. Traits Endpoints

Explore and analyze traits in the MR-KG database.

### 2.1 List Traits

In [7]:
response = requests.get(
    f"{BASE_URL}/traits/",
    params={
        "page": 1,
        "page_size": 10,
        "order_by": "appearance_count",
        "order_desc": True,
    },
)
data = check_response(response)
pretty_print(data)


Status: 200
{
  "success": true,
  "timestamp": "2025-10-16T07:53:10.898085Z",
  "request_id": null,
  "data": [
    {
      "trait_index": 355,
      "trait_label": "Body mass index",
      "appearance_count": 1305
    },
    {
      "trait_index": 246,
      "trait_label": "Type 2 diabetes",
      "appearance_count": 1264
    },
    {
      "trait_index": 63,
      "trait_label": "Body mass index (BMI)",
      "appearance_count": 848
    },
    {
      "trait_index": 3,
      "trait_label": "Alzheimer's disease",
      "appearance_count": 683
    },
    {
      "trait_index": 23,
      "trait_label": "Coronary artery disease",
      "appearance_count": 616
    },
    {
      "trait_index": 29,
      "trait_label": "Hypertension",
      "appearance_count": 601
    },
    {
      "trait_index": 185,
      "trait_label": "Stroke",
      "appearance_count": 600
    },
    {
      "trait_index": 25,
      "trait_label": "Heart failure",
      "appearance_count": 589
    },
    {
      "tr

### 2.2 Search Traits

In [8]:
response = requests.get(
    f"{BASE_URL}/traits/search",
    params={"q": "diabetes", "page": 1, "page_size": 10},
)
data = check_response(response)
pretty_print(data)

if data.get("data"):
    print(f"\nFound {len(data['data'])} traits matching 'diabetes'")


Status: 200
{
  "success": true,
  "timestamp": "2025-10-16T07:53:14.513953Z",
  "request_id": null,
  "data": [
    {
      "trait_index": 246,
      "trait_label": "Type 2 diabetes",
      "appearance_count": 1264
    },
    {
      "trait_index": 506,
      "trait_label": "Diabetes",
      "appearance_count": 185
    },
    {
      "trait_index": 197,
      "trait_label": "Type 2 diabetes mellitus (T2DM)",
      "appearance_count": 180
    },
    {
      "trait_index": 1300,
      "trait_label": "Type 2 diabetes mellitus",
      "appearance_count": 175
    },
    {
      "trait_index": 122,
      "trait_label": "Type 1 diabetes",
      "appearance_count": 163
    },
    {
      "trait_index": 44,
      "trait_label": "Type 2 diabetes (T2D)",
      "appearance_count": 157
    },
    {
      "trait_index": 128,
      "trait_label": "type 2 diabetes",
      "appearance_count": 133
    },
    {
      "trait_index": 242,
      "trait_label": "Gestational diabetes mellitus (GDM)",
      "

### 2.3 Get Trait Details

Get comprehensive information about a specific trait.

In [9]:
trait_index = 1

response = requests.get(
    f"{BASE_URL}/traits/{trait_index}",
    params={
        "include_studies": True,
        "include_similar": True,
        "include_efo": True,
        "max_studies": 5,
        "max_similar": 5,
        "similarity_threshold": 0.5,
    },
)
data = check_response(response)

if data.get("data"):
    trait_data = data["data"]
    print(f"Trait: {trait_data['trait']['trait_label']}")
    print(f"\nStatistics:")
    pretty_print(trait_data.get("statistics", {}))
    print(f"\nNumber of studies: {len(trait_data.get('studies', []))}")
    print(
        f"Number of similar traits: {len(trait_data.get('similar_traits', []))}"
    )
    print(f"Number of EFO mappings: {len(trait_data.get('efo_mappings', []))}")


Status: 200
Trait: migraine

Statistics:
{
  "study_count": 27,
  "model_distribution": {
    "llama3-2": 14,
    "gpt-4-1": 7,
    "llama3": 5,
    "o4-mini": 1
  },
  "publication_years": {
    "2024": 7,
    "2023": 8,
    "2022": 6,
    "2021": 4,
    "2020": 2
  }
}

Number of studies: 5
Number of similar traits: 5
Number of EFO mappings: 5


### 2.4 Get Trait Studies

In [10]:
trait_index = 1

response = requests.get(
    f"{BASE_URL}/traits/{trait_index}/studies",
    params={"page": 1, "page_size": 5},
)
data = check_response(response)

if data.get("data"):
    print(f"Studies for trait {trait_index}:")
    for study in data["data"]:
        print(f"\n- PMID: {study['pmid']}")
        print(f"  Model: {study['model']}")
        print(f"  Title: {study.get('title', 'N/A')}")
        print(f"  Journal: {study.get('journal', 'N/A')}")


Status: 200
Studies for trait 1:

- PMID: 38657649
  Model: llama3
  Title: Exploring the Two-Way Link between Migraines and Venous Thromboembolism: A Bidirectional Two-Sample Mendelian Randomization Study.
  Journal: Thrombosis and haemostasis

- PMID: 38657649
  Model: llama3-2
  Title: Exploring the Two-Way Link between Migraines and Venous Thromboembolism: A Bidirectional Two-Sample Mendelian Randomization Study.
  Journal: Thrombosis and haemostasis

- PMID: 38149430
  Model: llama3-2
  Title: Exploring the role of gut microbiota in migraine risk: a two-sample Mendelian randomization study.
  Journal: Scandinavian journal of gastroenterology

- PMID: 38435140
  Model: llama3-2
  Title: Causal Associations of Modifiable Risk Factors With Migraine: Evidence From Mendelian Randomization Analysis.
  Journal: Cureus

- PMID: 38650934
  Model: gpt-4-1
  Title: Genetic causal relationship between immune diseases and migraine: a Mendelian randomization study.
  Journal: Frontiers in immun

### 2.5 Find Similar Traits

In [11]:
trait_index = 1

response = requests.get(
    f"{BASE_URL}/traits/{trait_index}/similar",
    params={"max_results": 10, "similarity_threshold": 0.3},
)
data = check_response(response)

if data.get("data"):
    print(f"Similar traits to trait {trait_index}:")
    for item in data["data"][:5]:
        print(f"\n- Trait Index: {item['result_index']}")
        print(f"  Label: {item['result_label']}")
        print(f"  Similarity: {item['similarity_score']:.4f}")


Status: 200
Similar traits to trait 1:


KeyError: 'result_index'

### 2.6 Get EFO Mappings

In [12]:
trait_index = 1

response = requests.get(
    f"{BASE_URL}/traits/{trait_index}/efo-mappings",
    params={"max_results": 5, "similarity_threshold": 0.3},
)
data = check_response(response)

if data.get("data"):
    print(f"EFO mappings for trait {trait_index}:")
    for item in data["data"]:
        print(f"\n- EFO ID: {item['result_index']}")
        print(f"  Label: {item['result_label']}")
        print(f"  Similarity: {item['similarity_score']:.4f}")


Status: 200
EFO mappings for trait 1:


KeyError: 'result_index'

### 2.7 Traits Overview Statistics

In [13]:
response = requests.get(f"{BASE_URL}/traits/stats/overview")
data = check_response(response)

if data.get("data"):
    overview = data["data"]
    print(f"Total traits: {overview['total_traits']}")
    print(f"Total appearances: {overview['total_appearances']}")
    print(f"Average appearances: {overview['average_appearances']}")
    print(f"\nTop 5 traits:")
    for trait in overview["top_traits"][:5]:
        print(
            f"- {trait['trait_label']}: {trait['appearance_count']} appearances"
        )


Status: 200
Total traits: 25385
Total appearances: 114276
Average appearances: 4.5

Top 5 traits:
- Body mass index: 1305 appearances
- Type 2 diabetes: 1264 appearances
- Body mass index (BMI): 848 appearances
- Alzheimer's disease: 683 appearances
- Coronary artery disease: 616 appearances


### 2.8 Bulk Trait Retrieval

In [14]:
trait_indices = [1, 2, 3, 4, 5]

response = requests.post(f"{BASE_URL}/traits/bulk", json=trait_indices)
data = check_response(response)

if data.get("data"):
    print(f"Retrieved {len(data['data'])} traits:")
    for trait in data["data"]:
        print(f"- Index {trait['trait_index']}: {trait['trait_label']}")


Status: 200
Retrieved 5 traits:
- Index 1: migraine
- Index 2: migraine with aura
- Index 3: Alzheimer's disease
- Index 4: Parkinson's disease
- Index 5: amyotrophic lateral sclerosis


## 3. Studies Endpoints

Explore MR studies and their metadata.

### 3.1 List Studies

In [15]:
response = requests.get(
    f"{BASE_URL}/studies/",
    params={
        "page": 1,
        "page_size": 10,
        "order_by": "pub_date",
        "order_desc": True,
    },
)
data = check_response(response)

if data.get("data"):
    print(f"Retrieved {len(data['data'])} studies:")
    for study in data["data"][:5]:
        print(f"\n- ID: {study['id']}")
        print(f"  PMID: {study['pmid']}")
        print(f"  Model: {study['model']}")
        print(f"  Title: {study.get('title', 'N/A')[:80]}...")
        print(f"  Traits: {study['trait_count']}")


Status: 200
Retrieved 10 studies:

- ID: 11935
  PMID: 38737752
  Model: llama3-2
  Title: GWAS advancements to investigate disease associations and biological mechanisms....
  Traits: 11

- ID: 15100
  PMID: 38737752
  Model: gpt-4-1
  Title: GWAS advancements to investigate disease associations and biological mechanisms....
  Traits: 0

- ID: 21787
  PMID: 38737752
  Model: o4-mini
  Title: GWAS advancements to investigate disease associations and biological mechanisms....
  Traits: 0

- ID: 2007
  PMID: 38737752
  Model: llama3
  Title: GWAS advancements to investigate disease associations and biological mechanisms....
  Traits: 0

- ID: 14556
  PMID: 38545784
  Model: gpt-4-1
  Title: Evaluation of Plasma Biomarkers for Causal Association With Peripheral Artery Di...
  Traits: 1


### 3.2 Search Studies

In [16]:
response = requests.get(
    f"{BASE_URL}/studies/search",
    params={"q": "cardiovascular", "page": 1, "page_size": 5},
)
data = check_response(response)

if data.get("data"):
    print(f"Found {len(data['data'])} studies matching 'cardiovascular':")
    for study in data["data"]:
        print(f"\n- PMID: {study['pmid']}")
        print(f"  Title: {study.get('title', 'N/A')[:80]}...")


Status: 200
Found 5 studies matching 'cardiovascular':

- PMID: 38150519
  Title: Assessment of Subclinical Atherosclerosis in Asymptomatic People In Vivo: Measur...

- PMID: 38150519
  Title: Assessment of Subclinical Atherosclerosis in Asymptomatic People In Vivo: Measur...

- PMID: 38150519
  Title: Assessment of Subclinical Atherosclerosis in Asymptomatic People In Vivo: Measur...

- PMID: 38150519
  Title: Assessment of Subclinical Atherosclerosis in Asymptomatic People In Vivo: Measur...

- PMID: 38798395
  Title: Adverse pregnancy outcomes and coronary artery disease risk: A negative control ...


### 3.3 Get Study Details

In [17]:
study_id = 1

response = requests.get(
    f"{BASE_URL}/studies/{study_id}",
    params={"include_traits": True, "include_similar": True, "max_similar": 5},
)
data = check_response(response)

if data.get("data"):
    study_data = data["data"]
    study_info = study_data["study"]
    print(f"Study ID: {study_info['id']}")
    print(f"PMID: {study_info['pmid']}")
    print(f"Model: {study_info['model']}")
    if study_data.get("pubmed_data"):
        pubmed = study_data["pubmed_data"]
        print(f"\nTitle: {pubmed.get('title', 'N/A')}")
        print(f"Journal: {pubmed.get('journal', 'N/A')}")
        print(f"Publication Date: {pubmed.get('pub_date', 'N/A')}")
    print(f"\nTraits: {len(study_data.get('traits', []))}")
    print(f"Similar studies: {len(study_data.get('similar_studies', []))}")


Status: 503
Error: {"success":false,"timestamp":"2025-10-16T07:54:15.766114Z","request_id":"3636fb37-9ace-4bd2-a6a0-440ac2009aed","error":{"code":"HTTP_ERROR","message":"Database service unavailable","field":null,"context":null},"errors":null}


### 3.4 Studies Overview Statistics

In [18]:
response = requests.get(f"{BASE_URL}/studies/stats/overview")
data = check_response(response)

if data.get("data"):
    overview = data["data"]
    print(f"Total studies: {overview['total_studies']}")
    print(f"Unique PMIDs: {overview['unique_pmids']}")
    print(f"\nModel distribution:")
    for model, count in list(overview["model_distribution"].items())[:5]:
        print(f"- {model}: {count}")


Status: 200
Total studies: 26165


KeyError: 'unique_pmids'

## 4. Similarity Endpoints

Compute and analyze trait similarities.

### 4.1 Analyze Similarity for PMID-Model Combination

In [19]:
response = requests.get(
    f"{BASE_URL}/similarities/analyze",
    params={
        "pmid": "12345678",
        "model": "IVW",
        "max_results": 10,
        "min_similarity": 0.3,
        "similarity_type": "trait_profile",
    },
)
data = check_response(response)

if data.get("data"):
    analysis = data["data"]
    print("Similarity Analysis:")
    print(
        f"Query: PMID {analysis['query_combination']['pmid']}, "
        f"Model {analysis['query_combination']['model']}"
    )
    print(f"\nFound {len(analysis['similarities'])} similar combinations")
    pretty_print(analysis.get("summary", {}))


Status: 503
Error: {"success":false,"timestamp":"2025-10-16T07:54:24.928363Z","request_id":"637c3d07-6c67-4b9b-ad28-b7c06cd708e3","error":{"code":"HTTP_ERROR","message":"Database service unavailable","field":null,"context":null},"errors":null}


### 4.2 Vector Similarity Search

In [20]:
trait_index = 1

response = requests.post(
    f"{BASE_URL}/similarities/vector-search",
    json={
        "source_index": trait_index,
        "search_type": "trait",
        "max_results": 10,
        "min_similarity": 0.3,
    },
)
data = check_response(response)

if data.get("data"):
    print(f"Vector similarity results for trait {trait_index}:")
    for result in data["data"][:5]:
        print(f"\n- Index: {result['result_index']}")
        print(f"  Label: {result['result_label']}")
        print(f"  Similarity: {result['similarity_score']:.4f}")


Status: 404
Error: {"success":false,"timestamp":"2025-10-16T07:54:28.091749Z","request_id":"829851d7-084d-4004-8031-5c06358b7b23","error":{"code":"HTTP_ERROR","message":"Not Found","field":null,"context":null},"errors":null}


### 4.3 Batch Trait-to-EFO Mappings

In [21]:
trait_indices = [1, 2, 3]

response = requests.post(
    f"{BASE_URL}/similarities/batch-trait-to-efo",
    json={"trait_indices": trait_indices, "top_k": 3, "threshold": 0.3},
)
data = check_response(response)

if data.get("data"):
    print("Batch EFO mappings:")
    for mapping in data["data"]:
        print(f"\nTrait {mapping['trait_index']}: {mapping['trait_label']}")
        print(f"  EFO mappings: {len(mapping['efo_mappings'])}")
        for efo in mapping["efo_mappings"][:2]:
            print(f"  - {efo['result_label']}: {efo['similarity_score']:.4f}")


Status: 404
Error: {"success":false,"timestamp":"2025-10-16T07:54:39.247812Z","request_id":"f5b4452f-30d0-41a0-b446-a01f238f41b1","error":{"code":"HTTP_ERROR","message":"Not Found","field":null,"context":null},"errors":null}


## 5. System Endpoints

System-level operations and statistics.

### 5.1 Database Statistics

In [22]:
response = requests.get(f"{BASE_URL}/system/database/stats")
data = check_response(response)

if data.get("data"):
    stats = data["data"]
    print("Database Statistics:")
    pretty_print(stats)


Status: 404
Error: {"success":false,"timestamp":"2025-10-16T07:54:43.399352Z","request_id":"fd8e2874-4f0f-4497-a812-943b0973ce3e","error":{"code":"HTTP_ERROR","message":"Not Found","field":null,"context":null},"errors":null}


### 5.2 Schema Information

In [23]:
response = requests.get(f"{BASE_URL}/system/database/schema")
data = check_response(response)

if data.get("data"):
    schema = data["data"]
    print("Database Schema:")
    print(
        f"\nVector Store Tables: {len(schema.get('vector_store', {}).get('tables', []))}"
    )
    print(
        f"Trait Profile Tables: {len(schema.get('trait_profile', {}).get('tables', []))}"
    )


Status: 404
Error: {"success":false,"timestamp":"2025-10-16T07:54:46.898769Z","request_id":"1db8e01a-2fe9-4bda-8485-7017e58fb1c5","error":{"code":"HTTP_ERROR","message":"Not Found","field":null,"context":null},"errors":null}


## 6. Advanced Use Cases

### 6.1 Find Traits and Their Similar Studies

In [24]:
search_term = "blood pressure"

response = requests.get(
    f"{BASE_URL}/traits/search",
    params={"q": search_term, "page": 1, "page_size": 3},
)
search_data = check_response(response)

if search_data.get("data"):
    print(f"Search results for '{search_term}':")
    for trait in search_data["data"]:
        trait_index = trait["trait_index"]
        print(f"\n\nTrait: {trait['trait_label']} (index: {trait_index})")
        print(f"Appearances: {trait['appearance_count']}")

        studies_response = requests.get(
            f"{BASE_URL}/traits/{trait_index}/studies",
            params={"page": 1, "page_size": 3},
        )
        studies_data = check_response(studies_response)

        if studies_data.get("data"):
            print(
                f"  Related studies: {studies_data['pagination']['total_items']}"
            )
            for study in studies_data["data"][:2]:
                print(
                    f"  - PMID {study['pmid']}: {study.get('title', 'N/A')[:60]}..."
                )


Status: 200
Search results for 'blood pressure':


Trait: Systolic blood pressure (index: 351)
Appearances: 460
Status: 200
  Related studies: 452
  - PMID 38800482: Uterine leiomyoma causes an increase in systolic blood press...
  - PMID 38716647: Risk Factors for Intracerebral Hemorrhage: Genome-Wide Assoc...


Trait: Diastolic blood pressure (index: 463)
Appearances: 289
Status: 200
  Related studies: 283
  - PMID 38800482: Uterine leiomyoma causes an increase in systolic blood press...
  - PMID 38716647: Risk Factors for Intracerebral Hemorrhage: Genome-Wide Assoc...


Trait: Blood pressure (index: 417)
Appearances: 231
Status: 200
  Related studies: 229
  - PMID 38591222: Exploring Genetic Associations of 3 Types of Risk Factors Wi...
  - PMID 38591222: Exploring Genetic Associations of 3 Types of Risk Factors Wi...


### 6.2 Compare Traits Using Similarity Metrics

In [25]:
trait_index_1 = 1
trait_index_2 = 2

print(f"Comparing traits {trait_index_1} and {trait_index_2}:\n")

for trait_idx in [trait_index_1, trait_index_2]:
    response = requests.get(f"{BASE_URL}/traits/{trait_idx}")
    data = check_response(response)
    if data.get("data"):
        trait_info = data["data"]["trait"]
        print(f"Trait {trait_idx}: {trait_info['trait_label']}")

response = requests.get(
    f"{BASE_URL}/traits/{trait_index_1}/similar",
    params={"max_results": 20, "similarity_threshold": 0.0},
)
similar_data = check_response(response)

if similar_data.get("data"):
    for item in similar_data["data"]:
        if item["result_index"] == trait_index_2:
            print(f"\nSimilarity score: {item['similarity_score']:.4f}")
            break
    else:
        print(f"\nTrait {trait_index_2} not found in similar traits")


Comparing traits 1 and 2:

Status: 200
Trait 1: migraine
Status: 200
Trait 2: migraine with aura
Status: 200


KeyError: 'result_index'

### 6.3 Explore Study Trait Profiles

In [26]:
study_id = 1

response = requests.get(
    f"{BASE_URL}/studies/{study_id}", params={"include_traits": True}
)
data = check_response(response)

if data.get("data"):
    study_data = data["data"]
    study_info = study_data["study"]

    print(f"Study {study_id} - PMID {study_info['pmid']}")
    print(f"Model: {study_info['model']}")

    if study_data.get("pubmed_data"):
        print(f"Title: {study_data['pubmed_data'].get('title', 'N/A')}")

    print(f"\nTrait Profile ({len(study_data['traits'])} traits):")
    for trait in study_data["traits"][:10]:
        print(
            f"- Trait {trait['trait_index']}: {trait['trait_label']} "
            f"(role: {trait['role']})"
        )


Status: 503
Error: {"success":false,"timestamp":"2025-10-16T07:54:59.557253Z","request_id":"4fbc4dba-efc9-47dc-947b-5acc813ec5cc","error":{"code":"HTTP_ERROR","message":"Database service unavailable","field":null,"context":null},"errors":null}


## 7. Error Handling Examples

### 7.1 Handle Non-existent Resources

In [27]:
non_existent_trait = 999999

response = requests.get(f"{BASE_URL}/traits/{non_existent_trait}")
print(f"Status Code: {response.status_code}")
if not response.ok:
    print(f"Error Response:")
    pretty_print(response.json())


Status Code: 503
Error Response:
{
  "success": false,
  "timestamp": "2025-10-16T07:55:56.334012Z",
  "request_id": "f01d1221-2b22-47d3-b1dd-b885848b76bb",
  "error": {
    "code": "HTTP_ERROR",
    "message": "Database service unavailable",
    "field": null,
    "context": null
  },
  "errors": null
}


### 7.2 Handle Invalid Parameters

In [28]:
response = requests.get(
    f"{BASE_URL}/traits/", params={"page": -1, "page_size": 10000}
)
print(f"Status Code: {response.status_code}")
if not response.ok:
    print(f"Validation Error:")
    pretty_print(response.json())


Status Code: 422
Validation Error:
{
  "success": false,
  "timestamp": "2025-10-16T07:56:00.130976Z",
  "request_id": "3d7c960d-4f47-45b3-875f-4b645b1ba648",
  "error": {
    "code": "VALIDATION_ERROR",
    "message": "Input should be greater than or equal to 1",
    "field": "query -> page",
    "context": {
      "type": "greater_than_equal"
    }
  },
  "errors": [
    {
      "code": "VALIDATION_ERROR",
      "message": "Input should be greater than or equal to 1",
      "field": "query -> page",
      "context": {
        "type": "greater_than_equal"
      }
    },
    {
      "code": "VALIDATION_ERROR",
      "message": "Input should be less than or equal to 1000",
      "field": "query -> page_size",
      "context": {
        "type": "less_than_equal"
      }
    }
  ]
}
